# Conecting & Configuring the SQLite

In [1]:
%load_ext sql


In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://'+'/data/sql-murder-mystery.db')

In [3]:
%sql engine

# Analysis

## Searching for the city and the crime

In [4]:
%%sql
SELECT city,
COUNT(city) AS crimes_committed
FROM crime_scene_report
GROUP BY city
ORDER BY COUNT(city) DESC
LIMIT 5;

Running query in 'sqlite:///data/sql-murder-mystery.db'

city,crimes_committed
SQL City,9
Murfreesboro,9
Toledo,8
Jersey City,8
Evansville,8


In [5]:
%%sql
SELECT date,
description
FROM crime_scene_report
WHERE city='SQL City' AND
type = 'murder';

Running query in 'sqlite:///data/sql-murder-mystery.db'

date,description
20180215,REDACTED REDACTED REDACTED
20180215,Someone killed the guard! He took an arrow to the knee!
20180115,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


In [6]:
%%sql
SELECT SUBSTR(DATE,1,4) || '-' ||
SUBSTR(DATE,5,2) || '-' ||
SUBSTR(DATE,7,2) AS formatted_date,
description
FROM crime_scene_report
WHERE city='SQL City' AND
type = 'murder' AND date(formatted_date) = '2018-01-15';

Running query in 'sqlite:///data/sql-murder-mystery.db'

formatted_date,description
2018-01-15,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


## Searching the witness and seeing their testimonies

In [7]:
%%sql
SELECT id,
name,
address_number,
address_street_name
FROM person
WHERE address_number = (SELECT MAX(address_number)
                        FROM person
                        WHERE address_street_name = 'Northwestern Dr') OR
        (address_street_name = 'Franklin Ave' AND 
        name LIKE '%Annabel%');


Running query in 'sqlite:///data/sql-murder-mystery.db'

id,name,address_number,address_street_name
14887,Morty Schapiro,4919,Northwestern Dr
16371,Annabel Miller,103,Franklin Ave


In [8]:
%%sql
SELECT person_id,
p.name,
transcript
FROM interview
INNER JOIN person AS P
ON person_id = p.id
WHERE person_id IN (14887,16371);


Running query in 'sqlite:///data/sql-murder-mystery.db'

person_id,name,transcript
14887,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


# Creating a view for person details

In [9]:
%%sql
DROP VIEW IF EXISTS person_details;

Running query in 'sqlite:///data/sql-murder-mystery.db'

In [10]:
%%sql
CREATE VIEW person_details AS
    SELECT d.id AS driver_license,
        p.id AS person_id,
        p.name,
        p.ssn,
        d.age,
        d.height,
        d.eye_color,
        d.hair_color,
        d.gender,
        d.car_make,
        d.car_model,
        d.plate_number,
        p.address_number,
        p.address_street_name,
        i.annual_income
    FROM drivers_license AS d 
    INNER JOIN person AS p
    ON d.id = p.license_id
    INNER JOIN income AS i
    ON p.ssn=i.ssn

Running query in 'sqlite:///data/sql-murder-mystery.db'

# Query, using the testemonies, to get the murderer

In [11]:
%%sql
WITH first_witness AS (
    SELECT id AS membership_id,
        person_id,
        membership_status
    FROM get_fit_now_member
    WHERE id LIKE '48Z%' AND membership_status = 'gold'),
    
    second_witness AS (
        SELECT membership_id,
            m.name,
            m.person_id,
            SUBSTR(check_in_date,1,4) || '-' ||
            SUBSTR(check_in_date,5,2) || '-' ||
            SUBSTR(check_in_date,7,2) AS formatted_date,
            check_in_time,
            check_out_time
        FROM get_fit_now_check_in
        LEFT JOIN get_fit_now_member AS m
        ON membership_id = m.id
        WHERE formatted_date='2018-01-09')

SELECT pd.person_id,
    pd.name,
    pd.driver_license,
    pd.ssn,
    pd.plate_number,
    w1.membership_id,
    w1.membership_status,
    w2.formatted_date AS date_of_second_witness
FROM person_details AS pd
INNER JOIN first_witness AS w1
ON pd.person_id = w1.person_id
INNER JOIN second_witness AS w2
ON pd.person_id = w2.person_id
WHERE pd.plate_number LIKE '%H42W%';

Running query in 'sqlite:///data/sql-murder-mystery.db'

person_id,name,driver_license,ssn,plate_number,membership_id,membership_status,date_of_second_witness
67318,Jeremy Bowers,423327,871539279,0H42W2,48Z55,gold,2018-01-09


# The interview that the suspect gave

In [12]:
%%sql 
SELECT 
i.person_id,
p.name,
i.transcript
FROM interview AS i
INNER JOIN person AS p
ON i.person_id = p.id
WHERE person_id = 67318

Running query in 'sqlite:///data/sql-murder-mystery.db'

person_id,name,transcript
67318,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


# The person behind the murder and their testemony(or the lack of)

In [13]:
%%sql
WITH suspect_confection_event AS (
        SELECT person_id,
        event_id,
        event_name,
        COUNT(person_id) AS times_in_2017_12
        FROM facebook_event_checkin
        WHERE event_name = 'SQL Symphony Concert' AND 
        (SUBSTR(date,1,4) = '2017') AND 
        (SUBSTR(date,5,2) = '12')
        GROUP BY person_id,event_id,event_name
        HAVING COUNT(person_id) = 3)
SELECT
    pd.driver_license,
    pd.person_id,
    pd.name,
    pd.age,
    pd.height,
    pd.eye_color,
    pd.hair_color,
    pd.gender,
    pd.car_make,
    pd.car_model,
    pd.plate_number,
    pd.annual_income,
    ws.event_name,
    ws.times_in_2017_12
FROM person_details AS pd
INNER JOIN suspect_confection_event AS ws
ON pd.person_id = ws.person_id
WHERE (pd.height >= 65 AND pd.height <= 67) AND
pd.gender = 'female' AND
pd.hair_color = 'red' AND
(pd.car_make = 'Tesla' AND pd.car_model = 'Model S')
;

Running query in 'sqlite:///data/sql-murder-mystery.db'

driver_license,person_id,name,age,height,eye_color,hair_color,gender,car_make,car_model,plate_number,annual_income,event_name,times_in_2017_12
202298,99716,Miranda Priestly,68,66,green,red,female,Tesla,Model S,500123,310000,SQL Symphony Concert,3


In [14]:
%%sql 
SELECT 
i.person_id,
p.name,
i.transcript
FROM interview AS i
INNER JOIN person AS p
ON i.person_id = p.id
WHERE person_id = 99716

Running query in 'sqlite:///data/sql-murder-mystery.db'

person_id,name,transcript


# Apendix - Context


<img src="schema.png"
     alt="Schema"
     style="float: left; margin-right: 10px;" />

## Apendix - Queries and Views

## Testemunies

In [15]:
%%sql
-- First witness
SELECT id AS membership_id,
    person_id,
    membership_status
FROM get_fit_now_member
WHERE id LIKE '48Z%' AND membership_status = 'gold'

Running query in 'sqlite:///data/sql-murder-mystery.db'

membership_id,person_id,membership_status
48Z7A,28819,gold
48Z55,67318,gold


In [16]:
%%sql
-- Second witness
SELECT membership_id,
    m.name,
    m.person_id,
    SUBSTR(check_in_date,1,4) || '-' ||
    SUBSTR(check_in_date,5,2) || '-' ||
    SUBSTR(check_in_date,7,2) AS formatted_date,
    check_in_time,
    check_out_time
FROM get_fit_now_check_in
LEFT JOIN get_fit_now_member AS m
ON membership_id = m.id
WHERE formatted_date='2018-01-09'


Running query in 'sqlite:///data/sql-murder-mystery.db'

membership_id,name,person_id,formatted_date,check_in_time,check_out_time
X0643,Shondra Ledlow,15247,2018-01-09,957,1164
UK1F2,Zackary Cabotage,28073,2018-01-09,344,518
XTE42,Sarita Bartosh,55662,2018-01-09,486,1124
1AE2H,Adriane Pelligra,10815,2018-01-09,461,944
6LSTG,Burton Grippe,83186,2018-01-09,399,515
7MWHJ,Blossom Crescenzo,31523,2018-01-09,273,885
GE5Q8,Carmen Dimick,92736,2018-01-09,367,959
48Z7A,Joe Germuska,28819,2018-01-09,1600,1730
48Z55,Jeremy Bowers,67318,2018-01-09,1530,1700
90081,Annabel Miller,16371,2018-01-09,1600,1700


### View created

In [17]:
%%sql
SELECT *
FROM person_details
LIMIT 10;


Running query in 'sqlite:///data/sql-murder-mystery.db'

driver_license,person_id,name,ssn,age,height,eye_color,hair_color,gender,car_make,car_model,plate_number,address_number,address_street_name,annual_income
993845,10000,Christoper Peteuil,747714076,46,59,black,green,male,Chrysler,Town & Country,557472,624,Bankhall Ave,31000
861794,10007,Kourtney Calderwood,477972044,54,74,black,white,female,BMW,M Roadster,3P6DMS,2791,Gustavus Blvd,24000
385336,10010,Muoi Cary,828638512,24,79,blue,green,female,Mercedes-Benz,CLS-Class,GM6Y5J,741,Northwestern Dr,14800
431897,10016,Era Moselle,614621061,22,75,amber,red,female,Ford,Ranger,461S1M,1987,Wood Glade St,47400
439509,10027,Antione Godbolt,491650087,56,72,blue,blue,male,Honda,Civic,35S8LN,2431,Zelham Dr,79300
920494,10034,Kyra Buen,332497972,56,69,blue,grey,female,Infiniti,G,6K774M,1873,Sleigh Dr,21700
278151,10039,Francesco Agundez,861079251,87,54,black,blue,male,Pontiac,Grand Am,0F3B3Z,736,Buswell Dr,22500
779002,10122,Alva Conkel,148521773,52,64,black,blue,male,Lotus,Exige,8OLS80,116,Diversey Circle,28700
449378,10127,Yessenia Fossen,314282107,48,64,blue,blonde,female,Ford,Escort,XBO1R0,3087,Ash St,21400
627292,10141,Brittney Garfield,914555313,64,64,amber,brown,female,Lincoln,Navigator,4LXD5F,2303,E Glen Park Ave,67600
